INDEXING & RETRIEVAL

In [1]:
!pip install -U python-terrier

In [2]:
import os
import pandas as pd
import pyterrier as pt
from tqdm.notebook import tqdm

if not pt.started():
    pt.init()

PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [3]:
def experiment_br(retrs, topics, qrels, labels):
  return pt.Experiment(retrs, topics, qrels, names=labels, eval_metrics=['map', 'ndcg_cut.10', 'P.10', 'mrt'])

In [4]:
"""
!rm -rf /content/sample_data
!rm -rf /content/index_sci
!rm -rf /content/index_vas
"""

'\n!rm -rf /content/sample_data\n!rm -rf /content/index_sci\n!rm -rf /content/index_vas\n'

[SCIDOCS](https://allenai.org/data/scidocs)

In [5]:
dataset_name_sci = 'irds:beir/scidocs'
path_index_sci = '/content/index_sci'
props_path = '/data.properties'
dataset_sci = pt.datasets.get_dataset(dataset_name_sci)

if not os.path.exists(path_index_sci):
    indexer_sci = pt.index.IterDictIndexer(path_index_sci)
    index_ref_sci = indexer_sci.index(dataset_sci.get_corpus_iter(), fields=['text', 'title'])
else:
    index_ref_sci = pt.IndexRef.of(path_index_sci + props_path)

index_sci = pt.IndexFactory.of(index_ref_sci)
topics_sci = dataset_sci.get_topics('text')
qrels_sci = dataset_sci.get_qrels()

BM25_sci = pt.BatchRetrieve(index_sci, wmodel="BM25")
DPH_sci  = pt.BatchRetrieve(index_sci, wmodel="DPH")
PL2_sci  = pt.BatchRetrieve(index_sci, wmodel="PL2")
DLM_sci  = pt.BatchRetrieve(index_sci, wmodel="DirichletLM")

beir/scidocs documents:   0%|          | 0/25657 [00:00<?, ?it/s]

In [6]:
experiment_br([BM25_sci, DPH_sci, PL2_sci, DLM_sci], topics_sci, qrels_sci, ['BM25', 'DPH', 'PL2', 'DLM'])

,name,map,ndcg_cut.10,P.10,mrt
0,BM25,0.110141,0.158157,0.0813,37.464956
1,DPH,0.103363,0.149946,0.0780,35.509359
2,PL2,0.110610,0.159576,0.0828,35.814066
3,DLM,0.092528,0.134022,0.0714,34.810082


[VASWANI](http://ir.dcs.gla.ac.uk/resources/test_collections/npl/)

In [7]:
dataset_name_vas = 'irds:vaswani'
path_index_vas = '/content/index_vas'
dataset_vas = pt.datasets.get_dataset(dataset_name_vas)

if not os.path.exists(path_index_vas):
    indexer_vas = pt.index.IterDictIndexer(path_index_vas)
    index_ref_vas = indexer_vas.index(dataset_vas.get_corpus_iter(), meta=('docno', 'text'))
else:
    index_ref_vas = pt.IndexRef.of(path_index_vas + props_path)

index_vas = pt.IndexFactory.of(index_ref_vas)
topics_vas = dataset_vas.get_topics()
qrels_vas = dataset_vas.get_qrels()

BM25_vas = pt.BatchRetrieve(index_vas, wmodel="BM25")
DPH_vas  = pt.BatchRetrieve(index_vas, wmodel="DPH")
PL2_vas  = pt.BatchRetrieve(index_vas, wmodel="PL2")
DLM_vas  = pt.BatchRetrieve(index_vas, wmodel="DirichletLM")

[INFO] [starting] building docstore
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:06, 332kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 7.7%| 164k/2.13M [00:00<00:03, 516kB/s] 

[INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [3.64MB/s]
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [3.51MB/s]
docs_iter: 100%|██████████████████████| 11429/11429 [00:01<00:00, 10286.53doc/s]
[INFO] [finished] docs_iter: [00:01] [11429doc] [10268.51doc/s]
[INFO] [finished] building docstore [1.12s]


vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [8]:
experiment_br([BM25_vas, DPH_vas, PL2_vas, DLM_vas], topics_vas, qrels_vas, ['BM25', 'DPH', 'PL2', 'DLM'])

,name,map,ndcg_cut.10,P.10,mrt
0,BM25,0.296517,0.446609,0.352688,80.392842
1,DPH,0.283614,0.434968,0.358065,65.594233
2,PL2,0.276257,0.424514,0.338710,62.175777
3,DLM,0.196005,0.297378,0.239785,67.076554


end of fun.